In [14]:
# shadril238
# processing raw data to combined data
!pip install pandasql

In [16]:
import pandas as pd
import sqlite3
import numpy as np
import math
import os
import sys

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
# Participant to process
USER = 'participant1'

# Collection dataset
# COLLECTION = '2-person'
COLLECTION = '10-person'

# Define train data path
TRAIN_DATA_PATH = f'/content/drive/MyDrive/ML DL/ABC2023/ABC2023-WearingOffChallenge/pd_train_dataset/{COLLECTION}'

# Define test data path
TEST_DATA_PATH = f'/content/drive/MyDrive/ML DL/ABC2023/ABC2023-WearingOffChallenge/pd_test_dataset'

# To check if the TRAIN_DATA_PATH exists
if os.path.exists(TRAIN_DATA_PATH):
    print('TRAIN_DATA_PATH exists!')
else:
    print('TRAIN_DATA_PATH does not exist!')

# To check if the TEST_DATA_PATH exists
if os.path.exists(TEST_DATA_PATH):
    print('TEST_DATA_PATH exists!')
else:
    print('TEST_DATA_PATH does not exist!')

# Desired interval
#   * 15min: 15 minutes
#   * 30min: 30 minutes
#   * 1h: 1 hour
#   * 15s: 15 seconds

TARGET_FREQ_AS_INT = 15     # 15|1
TARGET_FREQ_UNIT1 = 'min'   # min|s
TARGET_FREQ_UNIT2 = 'm'     # m|s
TARGET_FREQ = f'{TARGET_FREQ_AS_INT}{TARGET_FREQ_UNIT1}'

# Wearing-off columns for renaming where value is the output
WEARING_OFF_COLUMNS = {
  "Timestamp": "timestamp",
  "Wearing Off": "wearing_off",
  "started_at": "wo_start", "finished_at": "wo_end",
  "tremors": "wo_tremors",
  "slowdown": "wo_slowdown",
  "moodchange": "wo_moodchange",
  "rigidity": "wo_rigidity",
  "pain": "wo_pain",
  "impairment_hands": "wo_impairment_hands",
  "slow_thoughts": "wo_slow_thoughts",
  "anxiety": "wo_anxiety",
  "muscle_spasm": "wo_muscle_spasm"
}

# Drug intake columns for renaming where value is the output
DRUG_INTAKE_COLUMNS = {
  "started_at": "drug_intake_start",
  "finished_at": "drug_intake_end",
  "tremors": "drug_intake_tremors",
  "slowdown": "drug_intake_slowdown",
  "moodchange": "drug_intake_moodchange",
  "rigidity": "drug_intake_rigidity",
  "pain": "drug_intake_pain",
  "impairment_hands": "drug_intake_impairment_hands",
  "slow_thoughts": "drug_intake_slow_thoughts",
  "anxiety": "drug_intake_anxiety",
  "muscle_spasm": "drug_intake_muscle_spasm"
}

# Define test periods
test_horizons = {
  "participant1": ["2021-12-02 0:00", "2021-12-03 23:45"],
  "participant2": ["2021-11-28 0:00", "2021-11-29 23:45"],
  "participant3": ["2021-11-25 0:00", "2021-11-26 23:45"],
  "participant4": ["2021-12-06 0:00", "2021-12-07 7:15"],
  "participant5": ["2021-11-28 0:00", "2021-11-29 23:45"],
  "participant6": ["2021-12-06 0:00", "2021-12-07 23:45"],
  "participant7": ["2021-12-12 0:00", "2021-12-13 9:45"],
  "participant8": ["2021-12-23 0:00", "2021-12-24 23:45"],
  "participant9": ["2021-12-23 0:00", "2021-12-24 23:45"],
  "participant10": ["2021-12-23 0:00", "2021-12-24 23:45"],
}

# Convert test periods to DataFrame
test_horizons_df = pd.DataFrame(
  [[participant, test_start_date, test_end_date]
   for participant, (test_start_date, test_end_date) in test_horizons.items()],
  columns=['participant', 'test_start_date', 'test_end_date']
)

TRAIN_DATA_PATH exists!
TEST_DATA_PATH exists!


In [20]:
heart_rate = pd.read_excel(f'{TRAIN_DATA_PATH}/garmin.xlsx',
                           sheet_name='heart_rate', index_col='Timestamp'
                          ).query(f'participant == "{USER}"')

heart_rate_test = pd.read_excel(f'{TEST_DATA_PATH}/garmin.xlsx',
                                sheet_name='heart_rate', index_col='Timestamp'
                               ).query(f'participant == "{USER}"')

In [21]:
steps = pd.read_excel(f'{TRAIN_DATA_PATH}/garmin.xlsx',
                      sheet_name='steps', index_col='Timestamp'
                     ).query(f'participant == "{USER}"')

steps_test = pd.read_excel(f'{TEST_DATA_PATH}/garmin.xlsx',
                           sheet_name='steps', index_col='Timestamp'
                          ).query(f'participant == "{USER}"')

In [22]:
stress = pd.read_excel(f'{TRAIN_DATA_PATH}/garmin.xlsx',
                       sheet_name='stress', index_col='Timestamp'
                      ).query(f'participant == "{USER}"')

stress_test = pd.read_excel(f'{TEST_DATA_PATH}/garmin.xlsx',
                            sheet_name='stress', index_col='Timestamp'
                           ).query(f'participant == "{USER}"')

In [23]:
sleep = pd.read_excel(f'{TRAIN_DATA_PATH}/garmin.xlsx',
                      sheet_name='sleep', index_col='Calendar Date'
                     ).query(f'participant == "{USER}"')

sleep_test = pd.read_excel(f'{TEST_DATA_PATH}/garmin.xlsx',
                           sheet_name='sleep', index_col='Calendar Date'
                          ).query(f'participant == "{USER}"')

In [24]:
wearing_off_periods = pd.read_excel(
  f'{TRAIN_DATA_PATH}/wearing_off_periods.xlsx',
  sheet_name='wearing_off_periods', index_col='wearing_off_id'
).query(
  f'participant == "{USER}"'
).drop(columns=['participant'])

In [25]:
drug_intake_with_symptoms = pd.read_excel(
  f'{TRAIN_DATA_PATH}/drug_intake_with_symptoms.xlsx',
  sheet_name='drug_intake_with_symptoms', index_col='drug_intake_id'
).query(
  f'participant == "{USER}"'
).drop(columns=['participant'])

In [26]:
def complete_reference(df, target_column, set, freq='15s'):
  # Ensures the max date of train set is last available before test set
  train_max_date = pd.to_datetime(
    test_horizons_df.query(
      f'participant == "{USER}"'
    ).test_start_date,
    format='%Y-%m-%d %H:%M'
  ) - pd.Timedelta(freq)

  # Ensures the min date of test set is the start of test set
  test_min_date = pd.to_datetime(
    test_horizons_df.query(
      f'participant == "{USER}"'
    ).test_start_date,
    format='%Y-%m-%d %H:%M'
  )

  # Create reference dataframe based on the start date and end date.
  if set == 'train':
    reference = pd.DataFrame(
      index=pd.date_range(
        start=df.index.min(),
        end=train_max_date.max(),
        freq=freq,
        name='Timestamp'
      )
    )
  elif set == 'test':
    reference = pd.DataFrame(
      index=pd.date_range(
        start=test_min_date.min(),
        end=df.index.max(),
        freq=freq,
        name='Timestamp'
      )
    )
  else:
    # Base case to get the min and max from the data
    reference = pd.DataFrame(
      index=pd.date_range(
        start=df.index.min(),
        end=df.index.max(),
        freq=freq,
        name='Timestamp'
      )
    )

  # Resample by dataset's interval
  #   Merge reference with heart rate data
  #   Fill missing values based on dataset's handling of missing values
  df = reference.merge(
    df.resample(freq).agg({
      target_column: 'mean',
      'participant': 'first'
    }), on='Timestamp', how='left'
  ).fillna({
    target_column: -1,
    'participant': USER
  })
  return df


# Garmin's heart rate interval
heart_rate_freq = '15s'
heart_rate = complete_reference(heart_rate,
                                target_column='heart_rate',
                                set='train',
                                freq=heart_rate_freq)
heart_rate_test = complete_reference(heart_rate_test,
                                     target_column='heart_rate',
                                     set='test',
                                     freq=heart_rate_freq)

# Garmin's step interval
steps_freq = '15min'
steps = complete_reference(steps,
                           target_column='steps',
                           set='train',
                           freq=steps_freq)
steps_test = complete_reference(steps_test,
                                target_column='steps',
                                set='test',
                                freq=steps_freq)

# Garmin's stress interval
stress_freq = '3min'
stress = complete_reference(stress,
                            target_column='stress_score',
                            set='train',
                            freq=stress_freq)
stress_test = complete_reference(stress_test,
                                 target_column='stress_score',
                                 set='test',
                                 freq=stress_freq)

In [27]:
def transform_sleep_duration_to_sleep_per_day(df):
  # Compute duration in minutes
  df['duration'] = (df['End Time'] - df['Start Time']) / np.timedelta64(1, 'm')

  # Transform sleep data by sleep stage
  df = df.pivot_table(
    index='Calendar Date',
    columns='Sleep Type',
    values='duration',
    aggfunc='sum'
  )

  # Fill missing sleep stage with 0 minutes
  df = pd.DataFrame(df.to_records(),
                    columns=['Calendar Date',
                             'awake', 'light',
                             'rem', 'deep']
                    ).set_index('Calendar Date').fillna(0)

  # Set index as DateTimeIndex type
  df.index = pd.to_datetime(df.index)
  df.index.name = 'Timestamp'

  # Compute other sleep features
  df['nonrem_total'] = (df['deep'] + df['light'])
  df['total'] = (df['nonrem_total'] + df['rem'])
  df['nonrem_percentage'] = df['nonrem_total'] / df['total']
  df['sleep_efficiency'] = df['total'] / (df['total'] + df['awake'])

  # Ignore unmeasurable sleep classifcation
  if 'unmeasurable' in df.columns:
    df.drop(columns='unmeasurable', inplace=True)

  return df


sleep = transform_sleep_duration_to_sleep_per_day(sleep)
sleep_test = transform_sleep_duration_to_sleep_per_day(sleep_test)

In [28]:
# Train Set
start = [heart_rate.index.min(),
         steps.index.min(),
         stress.index.min(),
         sleep.index.min()]

end = [heart_rate.index.max(),
       steps.index.max(),
       stress.index.max(),
       sleep.index.max()]

# Get minimum and maximum date from start
start = min(start)
end = max(end)

print("Check min and max value from train set")
print(f"min date: {start}")
print(
  f'Heart rate: {heart_rate.index.min()}\n',
  f'Steps: {steps.index.min()}\n',
  f'Stress: {stress.index.min()}\n',
  f'Sleep: {sleep.index.min()}\n',
)

print(f"max date: {end}")
print(
  f'Heart rate: {heart_rate.index.max()}\n',
  f'Steps: {steps.index.max()}\n',
  f'Stress: {stress.index.max()}\n',
  f'Sleep: {sleep.index.max()}\n',
)

Check min and max value from train set
min date: 2021-11-25 00:00:00
Heart rate: 2021-11-25 00:37:15
 Steps: 2021-11-25 00:00:00
 Stress: 2021-11-25 00:00:00
 Sleep: 2021-11-26 00:00:00

max date: 2021-12-01 23:59:45
Heart rate: 2021-12-01 23:59:45
 Steps: 2021-12-01 23:45:00
 Stress: 2021-12-01 23:57:00
 Sleep: 2021-12-01 00:00:00



In [29]:
# Test Set
start_test = [heart_rate_test.index.min(),
              steps_test.index.min(),
              stress_test.index.min(),
              sleep_test.index.min()]

end_test = [heart_rate_test.index.max(),
            steps_test.index.max(),
            stress_test.index.max(),
            sleep_test.index.max()]

# Get minimum and maximum date from start
start_test = min(start_test)
end_test = max(end_test)

print("Check min and max value from test set")
print(f"min date: {start_test}")
print(
  f'Heart rate: {heart_rate_test.index.min()}\n',
  f'Steps: {steps_test.index.min()}\n',
  f'Stress: {stress_test.index.min()}\n',
  f'Sleep: {sleep_test.index.min()}\n',
)

print(f"max date: {end_test}")
print(
  f'Heart rate: {heart_rate_test.index.max()}\n',
  f'Steps: {steps_test.index.max()}\n',
  f'Stress: {stress_test.index.max()}\n',
  f'Sleep: {sleep_test.index.max()}\n',
)

Check min and max value from test set
min date: 2021-12-02 00:00:00
Heart rate: 2021-12-02 00:00:00
 Steps: 2021-12-02 00:00:00
 Stress: 2021-12-02 00:00:00
 Sleep: 2021-12-02 00:00:00

max date: 2021-12-03 23:59:45
Heart rate: 2021-12-03 23:59:45
 Steps: 2021-12-03 23:45:00
 Stress: 2021-12-03 23:57:00
 Sleep: 2021-12-03 00:00:00



In [30]:
# Create reference dataframe based on train set's start date and end date.
reference = pd.DataFrame(
  index=pd.date_range(
    start=start,
    end=end,
    freq=TARGET_FREQ,
    name='Timestamp'
  )
)

# Combine each Garmin dataset to reference
#   * mean: get the average
#   * first: get the first value i.e., the value on that time
#   * last: get the last value i.e., the value on the previous time
# For example,
#   .resample(target_freq).agg({
#     'heart_rate': 'mean',
#   })
garmin_data = reference.merge(
  # Resample heart rate (15sec) to target frequency (e.g., 15min)
  heart_rate.resample(TARGET_FREQ).agg({
    'heart_rate': 'mean',
  }), on='Timestamp', how='left'
).ffill()

garmin_data = garmin_data.merge(
  # Resample steps (15min) to target frequency (e.g., 15min)
  steps.resample(TARGET_FREQ).agg({
    'steps': 'mean',
  }), on='Timestamp', how='left'
).ffill()

garmin_data = garmin_data.merge(
  # Resample stress (3min) to target frequency (e.g., 15min)
  stress.resample(TARGET_FREQ).agg({
    'stress_score': 'mean',
  }), on='Timestamp', how='left'
).ffill()

garmin_data = garmin_data.merge(
  # Resample sleep (1d) to target frequency (e.g., 15min)
  sleep.resample(TARGET_FREQ).mean(), on='Timestamp', how='left'
).ffill()

print(garmin_data.head(7).index)
print(garmin_data.tail(7).index)

DatetimeIndex(['2021-11-25 00:00:00', '2021-11-25 00:15:00',
               '2021-11-25 00:30:00', '2021-11-25 00:45:00',
               '2021-11-25 01:00:00', '2021-11-25 01:15:00',
               '2021-11-25 01:30:00'],
              dtype='datetime64[ns]', name='Timestamp', freq=None)
DatetimeIndex(['2021-12-01 22:15:00', '2021-12-01 22:30:00',
               '2021-12-01 22:45:00', '2021-12-01 23:00:00',
               '2021-12-01 23:15:00', '2021-12-01 23:30:00',
               '2021-12-01 23:45:00'],
              dtype='datetime64[ns]', name='Timestamp', freq=None)


In [31]:
# Create reference dataframe based on test set's start date and end date.
reference = pd.DataFrame(
  index=pd.date_range(
    start=start_test,
    end=end_test,
    freq=TARGET_FREQ,
    name='Timestamp'
  )
)

# Combine each Garmin dataset to reference
#   * mean: get the average
#   * first: get the first value i.e., the value on that time
#   * last: get the last value i.e., the value on the previous time
# For example,
#   .resample(target_freq).agg({
#     'heart_rate': 'mean',
#   })
garmin_data_test = reference.merge(
  # Resample heart rate (15sec) to target frequency (e.g., 15min)
  heart_rate_test.resample(TARGET_FREQ).agg({
    'heart_rate': 'mean',
  }), on='Timestamp', how='left'
).ffill()

garmin_data_test = garmin_data_test.merge(
  # Resample steps (15min) to target frequency (e.g., 15min)
  steps_test.resample(TARGET_FREQ).agg({
    'steps': 'mean',
  }), on='Timestamp', how='left'
).ffill()

garmin_data_test = garmin_data_test.merge(
  # Resample stress (3min) to target frequency (e.g., 15min)
  stress_test.resample(TARGET_FREQ).agg({
    'stress_score': 'mean',
  }), on='Timestamp', how='left'
).ffill()

garmin_data_test = garmin_data_test.merge(
  # Resample sleep (1d) to target frequency (e.g., 15min)
  sleep_test.resample(TARGET_FREQ).mean(), on='Timestamp', how='left'
).ffill()

print(garmin_data_test.head().index)
print(garmin_data_test.tail().index)

DatetimeIndex(['2021-12-02 00:00:00', '2021-12-02 00:15:00',
               '2021-12-02 00:30:00', '2021-12-02 00:45:00',
               '2021-12-02 01:00:00'],
              dtype='datetime64[ns]', name='Timestamp', freq=None)
DatetimeIndex(['2021-12-03 22:45:00', '2021-12-03 23:00:00',
               '2021-12-03 23:15:00', '2021-12-03 23:30:00',
               '2021-12-03 23:45:00'],
              dtype='datetime64[ns]', name='Timestamp', freq=None)


In [32]:
# Train set

# Match wearing-off periods to garmin timestamp by left join
#   Left join when wearing-off period start and end time is between garmin timestamp

cond_join = '''
            select
              garmin.*,
              wearing_off_periods.*,
              case
                when wearing_off_periods.[started_at] is not null THEN 1
                else 0
              end as 'Wearing Off'
            from garmin_data as garmin
            left join wearing_off_periods
            on garmin.[Timestamp] BETWEEN
            wearing_off_periods.[started_at] AND wearing_off_periods.[finished_at]
            '''

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')

# Load the DataFrames into the SQLite database
garmin_data.to_sql('garmin_data', conn)
wearing_off_periods.to_sql('wearing_off_periods', conn)

# Execute the SQL query
combined_data = pd.read_sql_query(cond_join, conn)

# Close the database connection
conn.close()

# Update columns
combined_data = combined_data.rename(columns=WEARING_OFF_COLUMNS)

# Drop duplicates based on timestamp
combined_data.drop_duplicates(subset='timestamp', inplace=True)

# Set timestamp as index, of type DateTimeIndex
combined_data.set_index('timestamp', inplace=True)
combined_data.index = pd.to_datetime(combined_data.index)

print(combined_data)

                     heart_rate  steps  stress_score  awake  light   rem  \
timestamp                                                                  
2021-11-25 00:00:00         NaN    0.0          -1.0    NaN    NaN   NaN   
2021-11-25 00:15:00         NaN    0.0          -1.0    NaN    NaN   NaN   
2021-11-25 00:30:00   43.419355    0.0          -0.4    NaN    NaN   NaN   
2021-11-25 00:45:00   60.250000    0.0          12.4    NaN    NaN   NaN   
2021-11-25 01:00:00   10.600000    0.0          -1.0    NaN    NaN   NaN   
...                         ...    ...           ...    ...    ...   ...   
2021-12-01 22:45:00  116.116667  504.0          16.0    1.0  236.0  47.0   
2021-12-01 23:00:00  103.616667  138.0          15.2    1.0  236.0  47.0   
2021-12-01 23:15:00   93.400000    0.0          47.0    1.0  236.0  47.0   
2021-12-01 23:30:00   35.316667    0.0           8.6    1.0  236.0  47.0   
2021-12-01 23:45:00   -1.000000    0.0          -1.0    1.0  236.0  47.0   

           

In [33]:
# Test set

# Since there is no available wearing-off periods,
#   copy Garmin data and apply same process as above
combined_data_test = garmin_data_test.copy().reset_index() \
  .rename(columns=WEARING_OFF_COLUMNS)

# Drop duplicates based on timestamp
combined_data_test.drop_duplicates(subset='timestamp', inplace=True)

# Set timestamp as index, of type DateTimeIndex
combined_data_test.set_index('timestamp', inplace=True)
combined_data_test.index = pd.to_datetime(combined_data_test.index)

In [34]:
# Train set


# Add TARGET_FREQ to the start date
#   to get the start and end date of the wearing off period
#   e.g., 15min -> 15min after
drug_intake_with_symptoms['started_at_after_buffer'] = (
  drug_intake_with_symptoms['started_at'] +
    np.timedelta64(TARGET_FREQ_AS_INT, TARGET_FREQ_UNIT2)
)

cond_join = '''
            select
              combined_data.*,
              drug_intake_with_symptoms.*,
              case
                when drug_intake_with_symptoms.[started_at] is not null THEN 1
                else 0
              end as 'drug_intake'
            from combined_data
            left join drug_intake_with_symptoms
            on combined_data.[timestamp] BETWEEN
            drug_intake_with_symptoms.[started_at] AND
            drug_intake_with_symptoms.[started_at_after_buffer]
          '''
# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')

# Load the DataFrames into the SQLite database
combined_data.to_sql('combined_data', conn)
drug_intake_with_symptoms.to_sql('drug_intake_with_symptoms', conn)

# Execute the SQL query
combined_data = pd.read_sql_query(cond_join, conn)

# Close the database connection
conn.close()

# Merge combined data with drug intake times
#   Then, update columns
combined_data = combined_data.rename(columns=DRUG_INTAKE_COLUMNS)

# Drop duplicates based on timestamp
combined_data.drop_duplicates(subset='timestamp', inplace=True)
combined_data.drop(
  columns=['started_at_after_buffer'],
  inplace=True)

# Set timestamp as index, of type DateTimeIndex
combined_data.set_index('timestamp', inplace=True)
combined_data.index = pd.to_datetime(combined_data.index)

print(combined_data.columns)

Index(['heart_rate', 'steps', 'stress_score', 'awake', 'light', 'rem', 'deep',
       'nonrem_total', 'total', 'nonrem_percentage', 'sleep_efficiency',
       'wearing_off_id', 'wo_start', 'wo_end', 'wo_pain', 'wo_tremors',
       'wo_anxiety', 'wo_rigidity', 'wo_slowdown', 'wo_slow_thoughts',
       'wo_impairment_hands', 'wo_moodchange', 'wo_muscle_spasm',
       'wearing_off', 'drug_intake_id', 'drug_intake_start',
       'drug_intake_pain', 'drug_intake_tremors', 'drug_intake_anxiety',
       'drug_intake_rigidity', 'drug_intake_slowdown',
       'drug_intake_slow_thoughts', 'drug_intake_impairment_hands',
       'drug_intake_moodchange', 'drug_intake_muscle_spasm', 'drug_intake'],
      dtype='object')


In [35]:
wo_symptoms = ['wo_pain', 'wo_tremors', 'wo_anxiety', 'wo_rigidity',
               'wo_slowdown', 'wo_slow_thoughts', 'wo_impairment_hands',
               'wo_moodchange', 'wo_muscle_spasm']

drug_intake_symptoms = ['drug_intake_pain', 'drug_intake_tremors',
                        'drug_intake_anxiety', 'drug_intake_rigidity',
                        'drug_intake_slowdown', 'drug_intake_slow_thoughts',
                        'drug_intake_impairment_hands',
                        'drug_intake_moodchange', 'drug_intake_muscle_spasm']

symptoms = ['pain', 'tremors', 'anxiety', 'rigidity', 'slowdown', 'slow_thoughts',
            'impairment_hands', 'moodchange', 'muscle_spasm',
            'wearing_off_post_meds']


def generate_final_symptoms(row):
  """
  Generate a list of final symptoms based on the patient's symptoms.
  """
  values = []
  for wo_symptom, drug_intake_symptom in zip(wo_symptoms, drug_intake_symptoms):
    # If the patient did not take medication, use the "without medication" symptom.
    if math.isnan(row[drug_intake_symptom]):
      if row[wo_symptom] is None or math.isnan(row[wo_symptom]):
        values.append(0)
      else:
        values.append(row[wo_symptom])
    # Otherwise, use the "after medication" symptom.
    else:
      values.append(row[drug_intake_symptom])
  # Add the "wearing off" symptom if any of the symptoms are present.
  if sum(values) >= 1:
    values.append(1)
  else:
    values.append(0)
  return pd.Series(values)

In [36]:
# Train set
final_symptoms = combined_data.apply(
  lambda row: generate_final_symptoms(row), axis=1
)

final_symptoms.columns = symptoms
combined_data = combined_data.merge(final_symptoms, on='timestamp', how='left')

In [37]:
def combine_wearing_offs(n):
  '''
  Check if the sum of the symptoms is greater than 0.
  If so, return 1, else return 0.
  That means there is at least 1 symptom present.
  '''
  if n > 0:
    return 1
  else:
    return 0

In [38]:
# TRAIN SET
combined_data["final_wearing_off"] = (
  combined_data.wearing_off + combined_data.wearing_off_post_meds
).apply(
  lambda n: combine_wearing_offs(n)
).values

In [39]:
display(
  combined_data.head(5)
)

display(
  combined_data.columns
)

,heart_rate,steps,stress_score,awake,light,rem,deep,nonrem_total,total,nonrem_percentage,...,tremors,anxiety,rigidity,slowdown,slow_thoughts,impairment_hands,moodchange,muscle_spasm,wearing_off_post_meds,final_wearing_off
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-11-25 00:00:00,NaN,0.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2021-11-25 00:15:00,NaN,0.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2021-11-25 00:30:00,43.419355,0.0,-0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2021-11-25 00:45:00,60.250000,0.0,12.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2021-11-25 01:00:00,10.600000,0.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


Index(['heart_rate', 'steps', 'stress_score', 'awake', 'light', 'rem', 'deep',
       'nonrem_total', 'total', 'nonrem_percentage', 'sleep_efficiency',
       'wearing_off_id', 'wo_start', 'wo_end', 'wo_pain', 'wo_tremors',
       'wo_anxiety', 'wo_rigidity', 'wo_slowdown', 'wo_slow_thoughts',
       'wo_impairment_hands', 'wo_moodchange', 'wo_muscle_spasm',
       'wearing_off', 'drug_intake_id', 'drug_intake_start',
       'drug_intake_pain', 'drug_intake_tremors', 'drug_intake_anxiety',
       'drug_intake_rigidity', 'drug_intake_slowdown',
       'drug_intake_slow_thoughts', 'drug_intake_impairment_hands',
       'drug_intake_moodchange', 'drug_intake_muscle_spasm', 'drug_intake',
       'pain', 'tremors', 'anxiety', 'rigidity', 'slowdown', 'slow_thoughts',
       'impairment_hands', 'moodchange', 'muscle_spasm',
       'wearing_off_post_meds', 'final_wearing_off'],
      dtype='object')

In [40]:
# Post-processing

# Compute wearing-off duration only during wearing-off periods
# TRAIN SET
combined_data['wo_duration'] = (
  pd.to_datetime(combined_data.index) -
  pd.to_datetime(combined_data['wo_start'])
) / np.timedelta64(1, TARGET_FREQ_UNIT2)
combined_data['wo_duration'] = combined_data['wo_duration'].fillna(0)

combined_data.loc[:, ['final_wearing_off',
                      'wo_duration', 'wo_start']].iloc[123:139]

,final_wearing_off,wo_duration,wo_start
timestamp,,,
2021-11-26 06:45:00,0,0.0,None
2021-11-26 07:00:00,0,0.0,None
2021-11-26 07:15:00,1,5.0,2021-11-26 07:10:00
2021-11-26 07:30:00,1,20.0,2021-11-26 07:10:00
2021-11-26 07:45:00,1,35.0,2021-11-26 07:10:00
2021-11-26 08:00:00,1,50.0,2021-11-26 07:10:00
2021-11-26 08:15:00,0,0.0,None
2021-11-26 08:30:00,0,0.0,None
2021-11-26 08:45:00,0,0.0,None


In [41]:
# # Compute wearing-off duration until the next reported wearing-off

# # TRAIN SET
# combined_data['wo_duration'] = (
#   pd.to_datetime(combined_data.index) -
#   pd.to_datetime(combined_data['wo_start'])
# ) / np.timedelta64(1, TARGET_FREQ_UNIT2)

# gid = combined_data['wo_duration'].notnull().cumsum()
# dg = combined_data.groupby(gid)
# base = dg['wo_duration'].transform('last')
# combined_data['wo_duration'] = (base + (dg.cumcount()) * TARGET_FREQ_AS_INT)

# combined_data.loc[:, ['final_wearing_off',
#                       'wo_duration', 'wo_start']].iloc[123:139]

In [42]:
# Compute time from last drug intake

# TRAIN SET
combined_data['time_from_last_drug_taken'] = (
  pd.to_datetime(combined_data.index) -
  pd.to_datetime(combined_data['drug_intake_start'])
) / np.timedelta64(1, TARGET_FREQ_UNIT2)

# Then, fill records after drug intake report by adding target frequency
gid = combined_data['time_from_last_drug_taken'].notnull().cumsum()
dg = combined_data.groupby(gid)
base = dg['time_from_last_drug_taken'].transform('last')
combined_data['time_from_last_drug_taken'] = (
  base + (dg.cumcount()) * TARGET_FREQ_AS_INT)

if combined_data['time_from_last_drug_taken'].isna().any():
  combined_data['time_from_last_drug_taken'] = \
    combined_data['time_from_last_drug_taken'].fillna(0)

combined_data.loc[:, ['drug_intake_id', 'drug_intake_start',
                      'time_from_last_drug_taken']].iloc[40:70]

,drug_intake_id,drug_intake_start,time_from_last_drug_taken
timestamp,,,
2021-11-25 10:00:00,1023.0,2021-11-25 09:54:00,6.0
2021-11-25 10:15:00,NaN,None,21.0
2021-11-25 10:30:00,NaN,None,36.0
2021-11-25 10:45:00,NaN,None,51.0
2021-11-25 11:00:00,1024.0,2021-11-25 11:00:00,0.0
2021-11-25 11:15:00,1024.0,2021-11-25 11:00:00,15.0
2021-11-25 11:30:00,NaN,None,30.0
2021-11-25 11:45:00,NaN,None,45.0
2021-11-25 12:00:00,NaN,None,60.0


In [43]:
# TRAIN SET
combined_data['timestamp_hour'] = combined_data.index.hour
combined_data['timestamp_dayofweek'] = combined_data.index.dayofweek

In [44]:
# TEST SET
combined_data_test['timestamp_hour'] = combined_data_test.index.hour
combined_data_test['timestamp_dayofweek'] = combined_data_test.index.dayofweek

In [45]:
# TRAIN SET

# Fix timestamp format
date_time = pd.to_datetime(combined_data.index, format='%d.%m.%Y %H:%M:%S')

# Convert to timestamp
timestamp_s = date_time.map(pd.Timestamp.timestamp)

# Get seconds per day
day = 24 * 60 * 60
# Get seconds per year
year = 365.2425 * day

# Get sine(), cosine() for hour-feature
combined_data['timestamp_hour_sin'] = np.sin(timestamp_s * (2 * np.pi / day))
combined_data['timestamp_hour_cos'] = np.cos(timestamp_s * (2 * np.pi / day))

In [46]:
# TEST SET

# Fix timestamp format
date_time = pd.to_datetime(combined_data_test.index, format='%d.%m.%Y %H:%M:%S')

# Convert to timestamp
timestamp_s = date_time.map(pd.Timestamp.timestamp)

# Get seconds per day
day = 24 * 60 * 60
# Get seconds per year
year = 365.2425 * day

# Get sine(), cosine() for hour-feature
combined_data_test['timestamp_hour_sin'] = np.sin(
    timestamp_s * (2 * np.pi / day))
combined_data_test['timestamp_hour_cos'] = np.cos(
    timestamp_s * (2 * np.pi / day))

In [47]:
# Save combined & processed dataset
# Choose features

# Garmin features
features = ['heart_rate', 'steps', 'stress_score',
            'awake', 'deep', 'light', 'rem',
            'nonrem_total', 'total', 'nonrem_percentage', 'sleep_efficiency']

# Additional features
features += ['timestamp_hour', 'timestamp_dayofweek',
             'timestamp_hour_sin', 'timestamp_hour_cos']

# Choices: wearing_off | wearing_off_post_meds | final_wearing_off
TARGET_COLUMN = 'final_wearing_off'
features.append(TARGET_COLUMN)

In [49]:
# Delete combined_data.xlsx if needed

# for_saving_file = f'{TRAIN_DATA_PATH}/combined_data.xlsx'
# # Delete file for_saving_file whether exists or not
# os.remove(for_saving_file)

# for_saving_file = f'{TEST_DATA_PATH}/combined_data.xlsx'
# # Delete file for_saving_file whether exists or not
# os.remove(for_saving_file)

In [50]:
# TRAIN SET


# Select features to include
for_saving = combined_data.loc[:, features].copy()

# Rename target column to 'wearing_off
for_saving.rename(columns={
  TARGET_COLUMN: 'wearing_off'
}, inplace=True)

# Add back the participant column
for_saving['participant'] = USER

# Save combined dataset to file
#  If the Excel file exists, append to the existing file
#  If the Excel file does not exist, create a new file
for_saving_file = f'{TRAIN_DATA_PATH}/combined_data.xlsx'
if os.path.exists(for_saving_file):
  with pd.ExcelWriter(for_saving_file, mode='a', if_sheet_exists='overlay') as writer:
    for_saving.reset_index().to_excel(
      writer, sheet_name='combined_dataset',
      startrow=writer.sheets['combined_dataset'].max_row,
      header=None, index=False
    )
else:
  with pd.ExcelWriter(for_saving_file) as writer:
    for_saving.to_excel(writer, sheet_name='combined_dataset')

In [51]:
# TEST SET

# Select features to include
combined_data_test[TARGET_COLUMN] = np.nan
combined_data_test = combined_data_test.reset_index().rename(
  columns={"Timestamp": "timestamp"}).set_index('timestamp')
for_saving = combined_data_test.loc[:, features].copy()

# Rename target column to 'wearing_off
for_saving.rename(columns={
  TARGET_COLUMN: 'wearing_off'
}, inplace=True)

# Add back the participant column
for_saving['participant'] = USER

# Save combined dataset to file
#  If the Excel file exists, append to the existing file
#  If the Excel file does not exist, create a new file

for_saving_file = f'{TEST_DATA_PATH}/combined_data.xlsx'
if os.path.exists(for_saving_file):
  with pd.ExcelWriter(for_saving_file, mode='a', if_sheet_exists='overlay') as writer:
    for_saving.reset_index().to_excel(
      writer, sheet_name='combined_dataset',
      startrow=writer.sheets['combined_dataset'].max_row,
      header=None, index=False
    )
else:
  with pd.ExcelWriter(for_saving_file) as writer:
    for_saving.to_excel(writer, sheet_name='combined_dataset')